# Finite Elasticity Part I

Authors: Jack S. Hale, Corrado Maurini.

Uses elements from https://jorgensd.github.io/dolfinx-tutorial/chapter2/hyperelasticity.html by Dokken and Wells under CC-BY 4.0

## Summary

In this notebook we will give an example of solving for the displacement field of a geometrically non-linear bar sagging under its own weight.

## Motivation

<img src="images/mazier_et_al_beam.jpeg" alt="drawing" width="500"/>

The above image shows image shows the *deformed configuration* of an initially straight silicone beam with cylindrical cross-section under its own weight. This material is very soft and quite dense. The resulting rotations and strains are large, so the assumptions made in a geometrically linear elastic model are no longer valid. For sensible predictions we must use the theory of finite elasticity. Source: Mazier et al. https://arxiv.org/abs/2102.13455

## Learning objectives

1. Briefly revisit the equations of non-linear elasticity.
1. Be able to express the Lagrangian functional of a geometrically non-linear elastic body in the Unified Form Language (UFL) of the FEniCS Project.
2. Use the automatic differentiation capabilities to derive symbolic expressions for the residual and Jacobian.
2. Understand and implement basic methods for solving non-linear problems that are available in DOLFINx.
3. See the difference in results between a geometrically linear and non-linear analysis.
4. Be aware of the possible effects and solutions to the problem of numerical volumetric locking.
5. Derive a stress measure automatically and output stresses.

## Possible extensions

1. Change boundary conditions to vertical beam under compression.
1. To a three-dimensional analysis.
2. Displacement-pressure (mixed) formulation to cure numerical locking.

In [137]:
import numpy as np

from mpi4py import MPI
from petsc4py import PETSc

import dolfinx
import dolfinx.io

import ufl

L = 1.0
H = 0.05
mesh = dolfinx.RectangleMesh(MPI.COMM_WORLD, [(0.0, 0.0, 0.0), (L, H, 0.0)], [100, 15])

V = dolfinx.VectorFunctionSpace(mesh, ("CG", 1))

def left(x):
    #print(x.shape)
    #print(x)
    is_close = np.isclose(x[0], 0.0)
    #print(is_close)
    return is_close

left_facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, left)
left_dofs = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, left_facets)

u_bc = dolfinx.Function(V)
with u_bc.vector.localForm() as loc:
    loc.set(0.0)
    
bcs = [dolfinx.DirichletBC(u_bc, left_dofs)]
    
u = dolfinx.Function(V)

# Identity tensor
d = len(u) # Spatial dimension
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J  = ufl.variable(ufl.det(F))

# Elasticity parameters
# nu is to be adjusted as exercise to observe locking.
E, nu = 1.0e4, 0.3
mu = dolfinx.Constant(mesh, E/(2*(1 + nu)))
lmbda = dolfinx.Constant(mesh, E*nu/((1 + nu)*(1 - 2*nu)))

# Body load in undeformed configuration
b = dolfinx.Constant(mesh, [0.0, -15.0])

# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2

# First Piola-Kirchhoff Stress
P = ufl.diff(psi, F)

# Stored strain energy density (compressible linear Hookean model)
# To be implemented as exercise having been given equations.
# Students should load into Paraview and comment on differences.
#epsilon = ufl.sym(ufl.grad(u))
#psi = mu * ufl.tr(epsilon*epsilon) + lmbda * (ufl.tr(epsilon))**2

dx = ufl.Measure("dx")
Pi = psi*dx - ufl.inner(b, u)*dx

F = ufl.derivative(Pi, u)
J = ufl.derivative(F, u)

# NOTE: Put this into a utilities.py file?
from typing import List
class NonlinearPDEProblem:
    """Nonlinear problem class.
    """
    def __init__(self, F: ufl.form.Form, J: ufl.form.Form, u: dolfinx.Function, bcs: List[dolfinx.DirichletBC]):
        """This class set up structures for solving a non-linear problem using Newton's method.
        
        Parameters
        ==========
        F: Residual.
        J: Jacobian.
        u: Solution.
        bcs: Dirichlet boundary conditions.
        """
        self.L = F
        self.a = J
        self.bcs = bcs

        # Create matrix and vector to be used for assembly
        # of the non-linear problem
        self.A = dolfinx.fem.create_matrix(self.a)
        self.b = dolfinx.fem.create_vector(self.L)

        
    def form(self, x: PETSc.Vec):
        """This function is called before the residual or Jacobian is computed inside the Newton step. 
        This is usually used to update ghost values.
        
        Parameters
        ==========
        x: Vector containing the latest solution.
        """
        x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

        
    def F(self, x: PETSc.Vec, b: PETSc.Vec):
        """Assemble the residual F into the vector b. 
        
        Parameters
        ==========
        x: Vector containing the latest solution.
        b: Vector to assemble the residual into.
        """
        # Zero the residual vector
        with b.localForm() as b_local:
            b_local.set(0.0)
        dolfinx.fem.assemble_vector(b, self.L)
        # Apply boundary conditions
        dolfinx.fem.apply_lifting(b, [self.a], [self.bcs], [x], -1.0)
        b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
        dolfinx.fem.set_bc(b, self.bcs, x, -1.0)

        
    def J(self, x: PETSc.Vec, A: PETSc.Mat):
        """Assemble the Jacobian matrix.
        
        Parameters
        ==========
        x: Vector containing the latest solution.
        A: Matrix to assemble the Jacobian into.
        """
        A.zeroEntries()
        dolfinx.fem.assemble_matrix(A, self.a, self.bcs)
        A.assemble()
     

problem = NonlinearPDEProblem(F, J, u, bcs)
solver = dolfinx.cpp.nls.NewtonSolver(MPI.COMM_WORLD)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"

# Set non-linear problem for Newton solver
solver.setF(problem.F, problem.b)
solver.setJ(problem.J, problem.A)
solver.set_form(problem.form)

num_its, converged = solver.solve(u.vector)
if converged:
    print(f"Converged in {num_its} iterations.")
else:
    print(f"Not converged.")

with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "displacement.xdmf", "w") as f:
    f.write_mesh(mesh)
    f.write_function(u)

# Solve for stresses
T = dolfinx.TensorFunctionSpace(mesh, ("DG", 0))

u = ufl.TrialFunction(T)
v = ufl.TestFunction(T)

a = ufl.inner(u, v)*dx
L = ufl.inner(P, v)*dx

problem = dolfinx.fem.LinearProblem(a, L)
P_h = problem.solve()

with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "stress.xdmf", "w") as f:
    f.write_mesh(mesh)
    f.write_function(P_h)

Converged in 11 iterations.
